In [1]:
import pandas as pd
import os
import h5py
import scipy.stats as stats
import numpy as np
from collections import defaultdict
from statsmodels.stats.multitest import multipletests

from BCBio import GFF
from Bio.Seq import Seq
from Bio.SeqUtils import GC
from Bio.SeqRecord import SeqRecord
from Bio.SeqFeature import SeqFeature, FeatureLocation

In [2]:
targets = [("H3K27ac", "_narrow"), ("H3K27me3", ""), ("H3K36me3", ""), 
           ("H3K4me1", "_narrow"), ("H3K4me2", "_narrow"), ("H3K4me3", "_narrow"), ("H3K79me2", ""), 
           ("H3K9ac", "_narrow"), ("H3K9me3", ""), ("H4K20me1", "")]
           #, ("methylation", "")]

In [3]:
def getLncRNAGeneSet():
    
    in_file = "../all_marks/gencode.v31.long_noncoding_RNAs.gff3"
    in_handle = open(in_file)

    limit_info = dict(
        gff_id = ['chr1', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 
     'chr19', 'chr2', 'chr20', 'chr21', 'chr22', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9', 'chrX', 'chrY'],
        gff_type = ["gene"])

    lncRNAgenes = []
    for rec in GFF.parse(in_handle, limit_info=limit_info):
        lncRNAgenes.append((rec.id, rec.features)) 

    in_handle.close()
    
    lncRNAgenesStartPositions = {
    g.id.split(".")[0]:[i, g.location.start.position, g.location.end.position, g.strand] for i, j in lncRNAgenes for g in j}
    
    return [i for i in lncRNAgenesStartPositions.keys() if i.startswith("ENSG")]

In [4]:
#Генерим бэкграунд для конкретного эксперимента: пересекаем наш список lncRNA(из которого мы выбирали)
#с экспериментальным списком генов, которые впринципе экспрессировались
#Если передаем несколько бэкграундов, значит хотим их объеденить
def getCommonBackground(*exp_backgrounds):
    lncRNASet = set(getLncRNAGeneSet())
    exp_background = None
    
    for exp in exp_backgrounds:
        lineList = [line.rstrip('\n') for line in open(exp)]
        lineList = [g for g in lineList if g.startswith("ENSG")]
        if exp_background is not None:
            exp_background = exp_background.union(set([r for l in [g.split(',') for g in lineList] for r in l]))
        else:
            exp_background = set([r for l in [g.split(',') for g in lineList] for r in l])
    
    return lncRNASet.intersection(exp_background)

In [5]:
#Возвращаем список мишеней для конкретного белка(в конкретной клетке), которые так же есть в бэкграунде
#Если белков несколько(могут быть из разных клеток), то сливаем их в один список
def getTargetsList(background, *protein_files):
    targets_list = None
    
    for p in protein_files:
        print("Get targets list for " + p)
        pretargets_list = pd.read_csv("../eCLIP/" + p, sep="\t")
        pretargets_list = [r.split(',') for r in pretargets_list[pretargets_list['Gene_ID'].str.startswith("ENSG")]['Gene_ID'].tolist()]
        
        if targets_list is not None:
            targets_list = targets_list.union(set([r for l in pretargets_list for r in l]))
        else:
            targets_list = set([r for l in pretargets_list for r in l])
            
    res = targets_list.intersection(background)
    
    print("Targets list len " + str(len(res)))
    return res

In [6]:
def getCorrsLncRNAsLists(target):
    def getCorrsLncRNAsLists(lncRNAnames, corrs):
        plus_corrs_idx = list(np.where(np.sum(corrs < 0, axis=1) == 0)[0])
        plus_hm_lncRNAs = lncRNAnames[plus_corrs_idx]
        minus_corrs_idx = list(np.where(np.sum(corrs > 0, axis=1) == 0)[0])
        minus_hm_lncRNAs = lncRNAnames[minus_corrs_idx]
        #mixture_idx = [i for i in range(data.shape[0]) if i not in (plus_corrs_idx + minus_corrs_idx)]
        #mixture_hm_lncRNAs = set(lncRNAnames[mixture_idx]).intersection(background)
   
        return {"general": lncRNAnames, "only + corrs": plus_hm_lncRNAs, "only - corrs": minus_hm_lncRNAs}
    

    print("makes count for " + target)
    if(target == "methylation"):
        corrs_lncRNAs = defaultdict(list)
        for i in range(0, 19):
            with h5py.File("../all_marks/" + target + "/lncRNA_Peaks_corrs/lncRNA_Peaks_Correlations_corrected_non_zero_" + str(i) + ".hdf5", 'r') as f:
                tmp = getCorrsLncRNAsLists(f['lncRNAs_names'][:], f['corrs_matrix'][:])
                corrs_lncRNAs["general"].append(tmp["general"])
                corrs_lncRNAs["only + corrs"].append(tmp["only + corrs"])
                corrs_lncRNAs["only - corrs"].append(tmp["only - corrs"])
    else:
        with h5py.File("../all_marks/" + target + "/lncRNA_Peaks_corrs/lncRNA_Peaks_Correlations_corrected_non_zero.hdf5", 'r') as f:
            corrs_lncRNAs = getCorrsLncRNAsLists(f['lncRNAs_names'][:], f['corrs_matrix'][:])
            
    return corrs_lncRNAs

In [18]:
corrs_lncRNAs = {}
for target in [("H3K27ac", "_narrow"), ("H3K27me3", "")]:
    corrs_lncRNAs[target[0]] = getCorrsLncRNAsLists(target[0])

makes count for H3K27ac
makes count for H3K27me3


In [7]:
def makeCounts(eclip, eclip_targets, background):

    res = []
    for target in targets:

        for list_type in  corrs_lncRNAs[target[0]].keys():
            type_list = set(corrs_lncRNAs[target[0]][list_type]).intersection(background)
            res.append((target[0], list_type, 
                        len(type_list), 
                        eclip, len(eclip_targets), 
                        len(type_list.intersection(set(eclip_targets)))))
            
                
    df = pd.DataFrame(res, columns =['Modification', 'Modification lncRNAs list type', 'Modification lncRNAs list count', 'RBP', 'RBP lncRNAs count', 'TP'])
    df['TN'] = len(background) - df['Modification lncRNAs list count'] - df['RBP lncRNAs count'] + df['TP']
    df['FP'] = df['Modification lncRNAs list count'] - df['TP']
    df['FN'] = df['RBP lncRNAs count'] - df['TP']
    
    return res, df

In [8]:
def count_p_values(df):
    pv_list = []
    oddsratio_list = []
    for index, row in df.iterrows():
        oddsratio, pv = stats.fisher_exact([[row['TP'], row['FP']], [row['FN'], row['TN']]], alternative='greater')
        pv_list.append(pv)
        oddsratio_list.append(oddsratio)
    
    df['p-value'] = pv_list
    df['oddsratio'] = oddsratio_list
    correct = multipletests(pv_list, alpha=0.05, method='fdr_bh')
    df['Correction'] = correct[0]
    
    return df

In [9]:
def getResults(background, proteins, proteins_names):
    res = []

    for protein, name in zip(proteins, proteins_names):
        if(isinstance(protein, list)):
            targets = getTargetsList(background, *protein)
        else:
            targets = getTargetsList(background, protein)

        counts = makeCounts(name, targets, background)
        res.append((name, count_p_values(counts)))
    
    concat = []
    for name, df in res:
        concat.append(df[df['Correction'] == True])
        
    return pd.concat(concat)

In [84]:
proteins = [f for f in os.listdir("../eCLIP/") if "txt" in f and "K562" in f]
proteins_names = [os.path.splitext(p)[0] for p in proteins]
background = getCommonBackground("../eCLIP/K562_background_genes.list")

res_k562, k562 = getResults(background, proteins, proteins_names)

Get targets list for TROVE2_K562.txt
Targets list len 10
Get targets list for SBDS_K562.txt
Targets list len 8
Get targets list for U2AF1_K562.txt
Targets list len 5
Get targets list for SAFB_K562.txt
Targets list len 343
Get targets list for KHDRBS1_K562.txt
Targets list len 90
Get targets list for SAFB2_K562.txt
Targets list len 541
Get targets list for PUS1_K562.txt
Targets list len 4
Get targets list for HNRNPL_K562.txt
Targets list len 340


In [85]:
proteins = [f for f in os.listdir("../eCLIP/") if "txt" in f and "HepG2" in f]
proteins_names = [os.path.splitext(p)[0] for p in proteins]

res_hepG2, hepG2 = getResults(getCommonBackground("../eCLIP/HepG2_background_genes.list"), proteins, proteins_names)

Get targets list for HNRNPL_HepG2.txt
Targets list len 144
Get targets list for TROVE2_HepG2.txt
Targets list len 4
Get targets list for SAFB_HepG2.txt
Targets list len 188
Get targets list for U2AF1_HepG2.txt
Targets list len 5


In [91]:
union_background = getCommonBackground("../eCLIP/K562_background_genes.list", 
                                          "../eCLIP/HepG2_background_genes.list")

proteins_names = ['HNRNPL', 'SAFB', 'TROVE2', 'U2AF1']
proteins = []

for name in proteins_names:
    proteins.append([f for f in os.listdir("../eCLIP/") if "txt" in f and name == os.path.splitext(f)[0].split('_')[0]])
    
res_union, union = getResults(union_background, proteins, [n + "_K562_HepG2_union" for n in proteins_names])

Get targets list for HNRNPL_HepG2.txt
Get targets list for HNRNPL_K562.txt
Targets list len 404
Get targets list for SAFB_K562.txt
Get targets list for SAFB_HepG2.txt
Targets list len 453
Get targets list for TROVE2_K562.txt
Get targets list for TROVE2_HepG2.txt
Targets list len 12
Get targets list for U2AF1_K562.txt
Get targets list for U2AF1_HepG2.txt
Targets list len 9


In [92]:
pd.concat([k562, hepG2, union]).to_csv("../eCLIP/first_exp_result.tsv", sep="\t", index=None)

In [14]:
pd.read_csv("../eCLIP/first_exp_result.tsv", sep="\t")

,Modification,Modification lncRNAs list type,Modification lncRNAs list count,RBP,RBP lncRNAs count,TP,TN,FP,FN,p-value,oddsratio,Correction
0,H3K27me3,only - corrs,77,SAFB2_K562,541,27,2201,50,514,7.583437e-04,2.312335,True
1,H3K27ac,general,764,SAFB_HepG2,188,76,2010,688,112,1.142265e-05,1.982454,True
2,H3K36me3,general,525,SAFB_HepG2,188,48,2221,477,140,5.974753e-03,1.596406,True
3,H3K36me3,only + corrs,86,SAFB_HepG2,188,12,2624,74,176,8.785641e-03,2.417690,True
4,H3K36me3,only - corrs,98,SAFB_HepG2,188,13,2613,85,175,9.890205e-03,2.283630,True
5,H3K4me1,general,766,SAFB_HepG2,188,66,1998,700,122,4.589168e-03,1.544122,True
6,H3K4me1,only + corrs,108,SAFB_HepG2,188,17,2607,91,171,4.699398e-04,2.848082,True
7,H3K27ac,general,876,SAFB_K562_HepG2_union,453,160,2289,716,293,2.370588e-07,1.745762,True
8,H3K27ac,only + corrs,187,SAFB_K562_HepG2_union,453,40,2858,147,413,8.213091e-04,1.883020,True
9,H3K27me3,general,622,SAFB_K562_HepG2_union,453,101,2484,521,352,7.194424e-03,1.368020,True


In [ ]:
#==================================================================================================================

In [ ]:
#Есть список генов с  сайтами посадки SAFB в K562
#Есть список генов с пиками конкретной метки, скоррелированными со списком конкретных РНК
#Есть список РНК с которыми вяжется SAFB
#1)Аннотировать сайты посадки SAFB - нашей аннотацией, потом все равно пересекать ее
#2)Сделать бэкграунд: пересечь все транскрибирующиеся гены в K562(по идее список, которым надо аннотировать сайты) с
#нашим списком(которым аннотировали пики)
#3)Сделать список lncRNA: пересечь список K562 SAFB с нашим списком lncRNA
#4)Есть список генов с сайтом, делаем список генов с пиком конкретной метки, скоррелированный со списком из
#предыдущего пункта
#)Пересекая эти списки делаем таблицу, потом считаем тест: по тесту на метку

In [96]:
target = "H3K27ac"

In [10]:
def makeCommonGeneSet():
    in_file = "../annotation/gencode.v31.annotation.gff3"
    in_handle = open(in_file)

    limit_info = dict(
        gff_id = ['chr1', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 
     'chr19', 'chr2', 'chr20', 'chr21', 'chr22', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9', 'chrX', 'chrY'],
        gff_type = ["gene"])

    gencode31_genes = []
    for rec in GFF.parse(in_handle, limit_info=limit_info):
        gencode31_genes.append(rec.features) 

    in_handle.close()
    
    gencode31_genes = [g.qualifiers['gene_id'][0].split('.')[0] for cr in gencode31_genes for g in cr]
    
    #гены K562
    lineList = [line.rstrip('\n') for line in open("../eCLIP/K562_background_genes.list")]
    lineList = [g for g in lineList if g.startswith("ENSG")]
    eCLIP_k562_genes = set([r for l in [g.split(',') for g in lineList] for r in l])
    
    common_genes = set(gencode31_genes).intersection(set(eCLIP_k562_genes))
    return common_genes

In [11]:
#Общие гены все(не только нкРНК)
common_genes = makeCommonGeneSet()

In [12]:
#lncRNAs background
common_lncRNAs = getCommonBackground("../eCLIP/K562_background_genes.list")

In [13]:
#Аннотация чипсека safb для K562
safb_anno = pd.read_csv("../eCLIP/SAFB_anno.csv", sep="\t")

In [14]:
#Список генов, которые входят в общий с нами список генов и в чипсик белка
safb_genes = set([f.split('.')[0] for f in safb_anno['feature']]).intersection(common_genes)

In [15]:
#нкРНК мишени у SAFB в K562(и которые могут быть в корреляциях)
k562_SAFB_rnas = getTargetsList(common_lncRNAs, "SAFB_K562.txt")

Get targets list for SAFB_K562.txt
Targets list len 343


In [16]:
def makeFisher(target):
    #нкРНК с корреляциями у H3K27ac(и которые могут быть в K562)
    target_rnas = set(corrs_lncRNAs[target]['general']).intersection(common_lncRNAs)
    
    #нкРНК, которые и имеют корреляции с меткой и есть в мишенях белка
    lncRNAs = k562_SAFB_rnas.intersection(target_rnas)
    
    pg_association = pd.read_csv("../all_marks/" + target + "/lncRNA_peaks_gene_association.tsv", sep="\t")
    
    #Список генов(общий список с генами белка), которые имеют корреляцию с меткой через нкРНК, которые так же мишени белка
    hm_genes = set([g.split('.')[0] for g in pg_association[pg_association['lncRNA'].isin(lncRNAs)]['gene'].unique()]).intersection(common_genes)

    TP = len(hm_genes.intersection(safb_genes))
    TN = len(common_genes) - len(safb_genes) - len(hm_genes) + TP
    FP = len(safb_genes) - TP
    FN = len(hm_genes) - TP
    
    return lncRNAs, hm_genes, stats.fisher_exact([[TP, FP], [FN, TN]], alternative='greater')

In [19]:
lncRNAs, hm_genes, fisher = makeFisher("H3K27ac")

In [20]:
fisher

(2.4309638997780683, 1.8633517278014684e-45)

In [21]:
lncRNAs, hm_genes, fisher = makeFisher("H3K27me3")

In [22]:
fisher

(1.2715115253958365, 1.7563058026120934e-12)

In [128]:
with open('../eCLIP/K562_SAFB_H3K27ac_lncRNAs.txt', mode='w', encoding='utf-8') as f:
    f.write('\n'.join(lncRNAs))

In [129]:
with open('../eCLIP/K562_SAFB_H3K27ac_genes.txt', mode='w', encoding='utf-8') as f:
    f.write('\n'.join(hm_genes.intersection(safb_genes)))